# Association Rule Mining

Apriori Algorithm  Implementation

In [12]:
# parameters for pruning
min_support = 2
min_conf    = 0.6 # 60% of confidance score

In [13]:
# transactions - array of array
# items - list
# itemset - dict
items = [1,2,3,4,5]

# dataset
transactions = [
    [1,3,4],    # transaction data R1
    [2,3,5],    # transaction data R2 ...
    [1,2,3,5],
    [2,5],
    [1,3,5]
]

# list required for processing the data
frequent_itemsets   = []
supports            = []
discarded           = []

### Helper Functions


In [14]:
def print_table(col1, col2):
    for i in range(len(col1)):
        print(f"{str(col1[i]).ljust(7)} {col2[i]}")

In [11]:
from itertools import combinations

def get_superset(_set, _n =0):
#     subsets = [set([])]
#     for nth in _set:
#         subsets.extend([s | {nth} for s in subsets])
#     return subsets
    subsets = []
    if _n == 1:
        subsets = [{s} for s in _set]
    elif _n > 1:
        combs = combinations(_set, _n)
        for comb in combs:
            subsets.append(set(comb))
        # subsets.extend(combinations(_set, r))
    else:
        for r in range(len(_set)):
            combs = combinations(_set, r)
            for comb in combs:
                subsets.append(set(comb))
    print(f"get_superset({_n}):  {subsets}")
    return subsets

In [5]:

# calculate support
# calculate_support({1,2,3})
def calculate_support(item):
    _support = 0
    # print(f"calculate_support(item) = {item}")

    # iterate over dataset and count occurance `item`
    for i in range(len(transactions)):
        # if set(transactions[i]).issubset(set(item)):
        if set(item).issubset(set(transactions[i])):
            # print(f"Found {set(item)} in {set(transactions[i])}")
            _support += 1
    return _support

In [6]:
# find items in itemset
def get_items(_itemset):
    items_in_itemset = []
    for _items in _itemset:
        for _item in list(_items):
            if _item not in items_in_itemset:
                items_in_itemset.append(_item)
    return items_in_itemset


### generate the frequent itemsets

In [7]:
def get_itemsetset(n):
    frequent_itemsets.append([])
    supports.append([])
    items_in_itemset = get_items(frequent_itemsets[n-1])

    print(f"Items in n-1={n-1} FrequentItemSet {items_in_itemset}")

    # get the superset
    sets = get_superset(items_in_itemset, n)

    # filter the itemset less than sn'th size
    _itemsets = []
    # purning
    for s in sets:
        if len(s) == n and s not in discarded:
            _itemsets.append(s)

    # without purning
    # _itemsets = [s for s in sets if len(s) == n]

    # calculate support and check the threshold
    # update the exclude list
    for _item in _itemsets:
        support_count = calculate_support(_item)
        if support_count >= min_support:
            frequent_itemsets[n].append(_item)
            supports[n].append(support_count)
        else: #exclude
            discarded.append(_item)
    print_table(frequent_itemsets[n], supports[n])

    return frequent_itemsets[n]

In [8]:
def generate_frequent_itemset():
    n = 1
    while (True):
        print(f" ======== n = {n} =========")
        _itemsets = get_itemsetset(n)
        if(len(_itemsets) > 1): n += 1
        else: break
    # return max frequent itemset
    return n-1

### Init the Datastructures

In [9]:
# initialize the itemset and support
#    we used 0th index to reduce the condition
#     check due to 0-based index
def init():
    # get 1 itemset and calculate support
    frequent_itemsets.append([])
    supports.append([])
    # dummy init
    for item in items:
        frequent_itemsets[0].append({item})
        supports[0].append(0)


### Step 1: Most Frequent Item set generation

In [15]:

# init the lists
init()
# Step 1: most frequent items
max_itemset_number = generate_frequent_itemset()

 ======== n = 1 =========
Items in n-1=0 FrequentItemSet [1, 2, 3, 4, 5]
get_superset(1):  [{1}, {2}, {3}, {4}, {5}]
{1}     3
{2}     3
{3}     4
{5}     4
 ======== n = 2 =========
Items in n-1=1 FrequentItemSet [1, 2, 3, 5]
get_superset(2):  [{1, 2}, {1, 3}, {1, 5}, {2, 3}, {2, 5}, {3, 5}]
{1, 3}  3
{1, 5}  2
{2, 3}  2
{2, 5}  3
{3, 5}  3
 ======== n = 3 =========
Items in n-1=2 FrequentItemSet [1, 3, 5, 2]
get_superset(3):  [{1, 3, 5}, {1, 2, 3}, {1, 2, 5}, {2, 3, 5}]
{1, 3, 5} 2
{2, 3, 5} 2
 ======== n = 4 =========
Items in n-1=3 FrequentItemSet [1, 3, 5, 2]
get_superset(4):  [{1, 2, 3, 5}]


### Step 2: formulation of rules

In [16]:
#       get all the item sets of max frequent set
#       get the superset of frequent itemset
#           S -> (I - S) i.e. => S recommends (I-S)
#       if support(I)/support(S) >= min_conf value
for I in frequent_itemsets[max_itemset_number]:
    print('='*30)
    subsets = get_superset(I)
    print(f"Subsets of I {I} = {subsets}")
    # S -> (I - S) i.e. => S recommends (I-S)
    for S in subsets:
        I_S = I - S
        if len(I_S) <= 0: continue
        I_support = calculate_support(I)
        S_support = calculate_support(S)
        conf_score = I_support / S_support
        is_selected = 'SELECTED' if conf_score >= min_conf else 'REJECTED'
        print(f"{S} -> {I_S} with Confidance: {conf_score} {is_selected}")

get_superset(0):  [set(), {1}, {3}, {5}, {1, 3}, {1, 5}, {3, 5}]
Subsets of I {1, 3, 5} = [set(), {1}, {3}, {5}, {1, 3}, {1, 5}, {3, 5}]
set() -> {1, 3, 5} with Confidance: 0.4 REJECTED
{1} -> {3, 5} with Confidance: 0.6666666666666666 SELECTED
{3} -> {1, 5} with Confidance: 0.5 REJECTED
{5} -> {1, 3} with Confidance: 0.5 REJECTED
{1, 3} -> {5} with Confidance: 0.6666666666666666 SELECTED
{1, 5} -> {3} with Confidance: 1.0 SELECTED
{3, 5} -> {1} with Confidance: 0.6666666666666666 SELECTED
get_superset(0):  [set(), {2}, {3}, {5}, {2, 3}, {2, 5}, {3, 5}]
Subsets of I {2, 3, 5} = [set(), {2}, {3}, {5}, {2, 3}, {2, 5}, {3, 5}]
set() -> {2, 3, 5} with Confidance: 0.4 REJECTED
{2} -> {3, 5} with Confidance: 0.6666666666666666 SELECTED
{3} -> {2, 5} with Confidance: 0.5 REJECTED
{5} -> {2, 3} with Confidance: 0.5 REJECTED
{2, 3} -> {5} with Confidance: 1.0 SELECTED
{2, 5} -> {3} with Confidance: 0.6666666666666666 SELECTED
{3, 5} -> {2} with Confidance: 0.6666666666666666 SELECTED


# Working on Real dataset

The csv contains the Shopping cart entries

In [17]:
import pandas as pd

df = pd.read_csv("/content/basket_data.csv")
df.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import csv

rows = []
with open("/content/basket_data.csv", 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        rows.append(row)

for row in rows:
    while('' in row):
        row.remove('')

In [ ]:
rows[1]

## Task 1: Data Collect and Extract

In [19]:
# Flatten the list of lists and convert it into a set
items_name_list = set(item for sublist in rows for item in sublist)

print(len(items_name_list))

120


## Task 2: Transform (Coding)

In [ ]:
## Decoding the data
original_items = {i+1: item for i, item in enumerate(items_name_list)}
original_items_reversed = {item:i+1 for i, item in enumerate(items_name_list)}
original_items

In [21]:
# prepare dataset suitable for algoritms
items = [key for key in original_items]
transactions_all = [[original_items_reversed[key] for key in sublist] for sublist in rows]

## Task3: Run Algoritm

In [29]:
# parameters for pruning
min_support = 20
min_conf    = 0.5 # 50% of confidance score

# list required for processing the data
frequent_itemsets   = []
supports            = []
discarded           = []

In [26]:
len(transactions_all)

7501

In [ ]:
transactions = transactions_all
# init the lists
init()
# Step 1: most frequent items
max_itemset_number = generate_frequent_itemset()

 ======== n = 1 =========
Items in n-1=0 FrequentItemSet [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
get_superset(1):  [{1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}, {12}, {13}, {14}, {15}, {16}, {17}, {18}, {19}, {20}, {21}, {22}, {23}, {24}, {25}, {26}, {27}, {28}, {29}, {30}, {31}, {32}, {33}, {34}, {35}, {36}, {37}, {38}, {39}, {40}, {41}, {42}, {43}, {44}, {45}, {46}, {47}, {48}, {49}, {50}, {51}, {52}, {53}, {54}, {55}, {56}, {57}, {58}, {59}, {60}, {61}, {62}, {63}, {64}, {65}, {66}, {67}, {68}, {69}, {70}, {71}, {72}, {73}, {7

In [28]:
# rule generation
#       get all the item sets of max frequent set
#       get the superset of frequent itemset
#           S -> (I - S) i.e. => S recommends (I-S)
#       if support(I)/support(S) >= min_conf value
for I in frequent_itemsets[max_itemset_number]:
    print('='*30)
    subsets = get_superset(I)
    print(f"Subsets of I {I} = {subsets}")
    # S -> (I - S) i.e. => S recommends (I-S)
    for S in subsets:
        I_S = I - S
        if len(I_S) <= 0: continue
        I_support = calculate_support(I)
        S_support = calculate_support(S)
        conf_score = I_support / S_support
        is_selected = 'SELECTED' if conf_score >= min_conf else 'REJECTED'
        print(f"{S} -> {I_S} with Confidance: {conf_score} {is_selected}")

get_superset(0):  [set(), {64}, {79}]
Subsets of I {64, 79} = [set(), {64}, {79}]
set() -> {64, 79} with Confidance: 0.05265964538061592 REJECTED
{64} -> {79} with Confidance: 0.3213995117982099 REJECTED
{79} -> {64} with Confidance: 0.220917225950783 REJECTED
get_superset(0):  [set(), {64}, {113}]
Subsets of I {64, 113} = [set(), {64}, {113}]
set() -> {64, 113} with Confidance: 0.03919477403012932 REJECTED
{64} -> {113} with Confidance: 0.23921887713588283 REJECTED
{113} -> {64} with Confidance: 0.225114854517611 REJECTED
get_superset(0):  [set(), {87}, {79}]
Subsets of I {87, 79} = [set(), {87}, {79}]
set() -> {79, 87} with Confidance: 0.05092654312758299 REJECTED
{87} -> {79} with Confidance: 0.28338278931750743 REJECTED
{79} -> {87} with Confidance: 0.21364653243847875 REJECTED
get_superset(0):  [set(), {113}, {79}]
Subsets of I {113, 79} = [set(), {113}, {79}]
set() -> {113, 79} with Confidance: 0.05972536995067324 REJECTED
{113} -> {79} with Confidance: 0.3430321592649311 REJECTE

## Task 4: Interprate

Decode the values and interprate the result